In [1]:
import numpy as np
import pandas as pd

## Preprocessing Operations 

In [16]:
df = pd.read_csv("chemicals.csv")

c:\users\jocel\appdata\local\programs\python\python35\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
def preprocess_chemicals(dataset_name = "chemicals.csv", yearbound=2000):
    df = pd.read_csv(dataset_name, encoding = "ISO-8859-1")
    for i, c in enumerate(df["contaminant_level"].unique()):
        df.loc[df["contaminant_level"]==c, "contaminant_level"] = i
    df.loc[df["unit_measurement"]=="milligrams/L", "value"]*=1000
    df.loc[df["unit_measurement"]=="milligrams/L", "unit_measurement"]="micrograms/L"
    return df

In [3]:
def preprocess_droughts(dataset_name='droughts', year_bound=2000):
    dataset_path = dataset_name + '.csv'
    df = pd.read_csv(dataset_path, encoding = "ISO-8859-1")
    valid_start = list(df['valid_start'])
    valid_start = list(map(lambda x: int(x[0:4]), valid_start))
    df['valid_start'] = valid_start
    valid_end = list(df['valid_end'])
    valid_end = list(map(lambda x: int(x[0:4]), valid_end))
    df['valid_end'] = valid_end
    valid_dates = df['valid_start'] >= year_bound
    df = df[valid_dates]
    df.to_csv(dataset_name + '_processed.csv', index=False)
    df['year'] = list(df['valid_start'])
    return df

In [14]:
def preprocess_earnings(dataset_name='earnings', year_bound=2000):
    dataset_path = dataset_name + '.csv'
    df = pd.read_csv(dataset_path, encoding = "ISO-8859-1")
    df_mod = pd.DataFrame({'fips':df['fips'], 'county':df['county'], 'total_med':df['total_med'], 'year':df['year']})
    df.to_csv(dataset_name + '_processed.csv', index=False)
    possible_years = df['year'] >= year_bound
    df = df[possible_years]
    return df

In [9]:
def preprocess_education_attainment(dataset_name="education_attainment", yearbound=2000):
    df = pd.read_csv(dataset_name + '.csv', encoding = "ISO-8859-1")
    year = list(df['year'])
    year = list(map(lambda x: int(x[0:4]), year))
    df['year'] = year
    df = df.loc[df["year"]>=yearbound]
    return df

In [6]:
def preprocess_industry_occupation(dataset_name='industry_occupation', year_bound=2000):
    dataset_path = dataset_name + '.csv'
    df = pd.read_csv(dataset_path, encoding = "ISO-8859-1")
    df = df[df['year'] > year_bound]
    df.to_csv(dataset_name + '_processed.csv', index=False)
    return df


In [7]:
def preprocess_water_usage(dataset_name = "water_usage", yearbound=2000):
    df = pd.read_csv("water_usage.csv", encoding = "ISO-8859-1")
    df = df.drop(['ps_groundwater', 'ps_surfacewater'], axis=1)
    return df

In [8]:
preprocess_chemicals().to_csv("chemicals_processed.csv", index=False)
preprocess_droughts().to_csv("droughts_processed.csv", index=False)
preprocess_earnings().to_csv("earnings_processed.csv", index=False)


c:\users\jocel\appdata\local\programs\python\python35\lib\site-packages\IPython\core\interactiveshell.py:2903: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [10]:
preprocess_education_attainment().to_csv("education_attainment_processed.csv", index=False)

In [9]:
preprocess_industry_occupation().to_csv("industry_occupation_processed.csv", index=False)
preprocess_water_usage().to_csv("water_usage_processed.csv", index=False)

## Joining Operations

In [9]:
'''
@param df : pandas dataframe
@param other_df : another pandas dataframe
@reurn merge of two dataframes
'''
def join_two_datasets(df, other_df, fields):

    df['fips'] = df['fips'].astype(int)
    other_df['fips'] = other_df['fips'].astype(int)
    return pd.merge(df, other_df, on=fields)

'''
@return dictionary mapping name to pandas data frame
'''
def generate_dfs():
    all_dfs = {}
    all_dfs['water_usage'] = preprocess_water_usage()
    all_dfs['industry_occupation'] = preprocess_industry_occupation()
    all_dfs['chemicals'] = preprocess_chemicals()
    all_dfs['droughts'] = preprocess_droughts()
    all_dfs['earnings'] = preprocess_earnings()
    all_dfs['education_attainment'] = preprocess_education_attainment()
    return all_dfs

In [10]:
all_dfs = generate_dfs()
water_usage_df = all_dfs['water_usage']
earnings_df = all_dfs['earnings']

In [11]:
new_df = join_two_datasets(water_usage_df, earnings_df, fields='fips')
new_df

,state,state_fips,county_x,county_fips,fips,year_x,population,ps_total,d_selfsupplied,d_totaluse,...,admin_sup,total_edu_health_social,edu_serv,health_social,total_arts_ent_acc_food,arts_ent_rec,acc_food_serv,other_ser,pub_admin,year_y
0,AL,1,Autauga County,1,1001,2010,54.571,48.222,6.349,3.54,...,31250,30306,34358,26839,11231,10272,11430,26279,46858,2010
1,AL,1,Baldwin County,3,1003,2010,182.265,153.463,28.802,13.66,...,23910,34506,37341,30968,14924,17414,14765,21440,38629,2010
2,AL,1,Barbour County,5,1005,2010,27.457,25.555,1.902,2.05,...,20179,22398,42264,20434,6473,10724,5972,16090,30871,2010
3,AL,1,Bibb County,7,1007,2010,22.915,21.279,1.636,1.84,...,13636,25474,36618,21632,16530,5114,20481,30956,33095,2010
4,AL,1,Blount County,9,1009,2010,57.322,44.464,12.858,3.87,...,27553,31931,36631,29958,7535,10050,7282,25326,34838,2010
5,AL,1,Bullock County,11,1011,2010,10.914,10.176,0.738,1.20,...,"2,500-",35269,37218,21250,12170,43012,10071,4278,34782,2010
6,AL,1,Butler County,13,1013,2010,20.947,17.599,3.348,1.62,...,26932,25313,28750,24970,6751,5294,6911,16979,28909,2010
7,AL,1,Calhoun County,15,1015,2010,118.572,112.390,6.182,9.91,...,25112,30625,35328,25211,9024,7434,9260,18696,47836,2010
8,AL,1,Chambers County,17,1017,2010,34.215,25.875,8.340,2.82,...,20046,26671,34176,23996,8924,67868,7279,16516,31898,2010
9,AL,1,Cherokee County,19,1019,2010,25.989,17.876,8.113,1.67,...,30729,35480,43429,25632,10230,"2,500-",10302,14091,40336,2010
